In [ ]:
import os
import sys

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.get_all_study_names()


In [ ]:
study_name = "xgridt_2024-11-22"
df = optimizer.get_study_trials_df(study_name)
df

In [ ]:
df_filtered = df[df["value"] > 0].sort_values("total_executors", ascending=False)
df_filtered

In [ ]:
import plotly.express as px

px.scatter(df_filtered, x="total_executors", y="sharpe_ratio", color="net_pnl", hover_data=["number"])

In [ ]:
from core.backtesting import BacktestingEngine
import json

import datetime

id_to_backtest = 7
resolution = "1m"
config = json.loads(df[df["number"] == id_to_backtest].iloc[0]["config"])
connector_name = config["connector_name"]
trading_pair = config["trading_pair"]
BACKTESTING_DAYS = 30
start_date = datetime.datetime.now() - datetime.timedelta(days=BACKTESTING_DAYS)
end_date = datetime.datetime.now()

backtesting_engine = BacktestingEngine(root_path=root_path, load_cached_data=False)
config = backtesting_engine.get_controller_config_instance_from_dict(config)

In [ ]:
bt_result = await backtesting_engine.run_backtesting(config=config, start=int(start_date.timestamp()), end=int(end_date.timestamp()), backtesting_resolution=resolution)

In [ ]:
print(bt_result.get_results_summary())
bt_result.get_backtesting_figure()

In [ ]:
from core.services.backend_api_client import BackendAPIClient

client = BackendAPIClient()

In [ ]:
bt_result.controller_config.dict()

In [ ]:
await client.add_controller_config(json.loads(bt_result.controller_config.json()))

In [ ]:
await client.deploy_script_with_controllers(
    bot_name="test_1",
    controller_configs=[bt_result.controller_config.id + ".yml"],
    image_name="dardonacci/hummingbot:latest",
    max_global_drawdown=50,
    max_controller_drawdown=20,
)